<a href="https://colab.research.google.com/github/keerthi612004/freelancer-assignment/blob/main/freelancerassignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio networkx matplotlib pandas scipy python-docx --quiet
import gradio as gr
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from docx import Document
from docx.shared import Pt, Inches
import tempfile
import matplotlib.patches as mpatches

 # Added for legend
def format_table(doc, df, title):
    doc.add_heading(title, level=2)
    table = doc.add_table(rows=1, cols=len(df.columns))
    table.style = 'Table Grid'
    hdr_cells = table.rows[0].cells
    for i, column_name in enumerate(df.columns):
        hdr_cells[i].text = str(column_name)
        hdr_cells[i].paragraphs[0].runs[0].font.bold = True
    for _, row in df.iterrows():
        row_cells = table.add_row().cells
        for i, value in enumerate(row):
            row_cells[i].text = str(value)
            for paragraph in row_cells[i].paragraphs:
                for run in paragraph.runs:
                    run.font.size = Pt(9)

def format_cost_matrix_table(doc, cost_matrix):
    doc.add_heading(" Cost Matrix", level=2)
    table = doc.add_table(rows=len(cost_matrix)+1, cols=len(cost_matrix[0])+1)
    table.style = 'Table Grid'
    table.cell(0, 0).text = ""
    for j in range(len(cost_matrix[0])):
        table.cell(0, j+1).text = f"T{j+1}"
    for i in range(len(cost_matrix)):
        table.cell(i+1, 0).text = f"F{i+1}"
        for j in range(len(cost_matrix[0])):
            table.cell(i+1, j+1).text = f"{cost_matrix[i][j]:.2f}"

def optimize_and_schedule(freelancer_file, task_file, budget):
    freelancers = pd.read_csv(freelancer_file.name)
    tasks = pd.read_csv(task_file.name)

    num_freelancers = len(freelancers)
    num_tasks = len(tasks)
    max_dim = max(num_freelancers, num_tasks)

    cost_matrix = np.full((max_dim, max_dim), 10000.0)
    for i in range(num_freelancers):
        for j in range(num_tasks):
            f = freelancers.iloc[i]
            t = tasks.iloc[j]
            if f["Skill"].strip().lower() == t["Required Skill"].strip().lower():
                cost_matrix[i][j] = f["Cost per Day"] / max(f["Experience (Years)"], 1)

    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    assignments = []
    total_cost = 0
    for i, j in zip(row_ind, col_ind):
        if i < num_freelancers and j < num_tasks and cost_matrix[i][j] < 9999:
            f = freelancers.iloc[i]
            t = tasks.iloc[j]
            cost = f["Cost per Day"] * t["Duration (days)"]
            total_cost += cost
            assignments.append({
                "Freelancer": f["Freelancer"],
                "Skill": f["Skill"],
                "Experience": f["Experience (Years)"],
                "Task ID": t["Task ID"],
                "Task Name": t["Task Name"],
                "Cost/Day": f["Cost per Day"],
                "Duration": t["Duration (days)"],
                "Total Cost": cost
            })

    df_assignments = pd.DataFrame(assignments)

    G = nx.DiGraph()
    for _, row in tasks.iterrows():
        G.add_node(row["Task ID"], name=row["Task Name"], duration=row["Duration (days)"])
    for _, row in tasks.iterrows():
        if pd.notna(row["Dependencies"]) and row["Dependencies"] != "":
            for dep in str(row["Dependencies"]).split(","):
                G.add_edge(dep.strip(), row["Task ID"])

    for node in nx.topological_sort(G):
        preds = list(G.predecessors(node))
        G.nodes[node]["ES"] = max([G.nodes[p]["EF"] for p in preds], default=0)
        G.nodes[node]["EF"] = G.nodes[node]["ES"] + G.nodes[node]["duration"]

    project_duration = max(G.nodes[n]["EF"] for n in G.nodes)

    for node in reversed(list(nx.topological_sort(G))):
        succs = list(G.successors(node))
        G.nodes[node]["LF"] = min([G.nodes[s]["LS"] for s in succs], default=project_duration)
        G.nodes[node]["LS"] = G.nodes[node]["LF"] - G.nodes[node]["duration"]

    critical_path = []
    schedule_data = []
    for node in G.nodes:
        slack = G.nodes[node]["LS"] - G.nodes[node]["ES"]
        schedule_data.append({
            "Task": node,
            "Name": G.nodes[node]["name"],
            "ES": G.nodes[node]["ES"],
            "EF": G.nodes[node]["EF"],
            "LS": G.nodes[node]["LS"],
            "LF": G.nodes[node]["LF"],
            "Slack": slack
        })
        if slack == 0:
            critical_path.append(node)

    # Plot graph with styles and legend
    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(14, 10))

    critical_edges = [(u, v) for u, v in G.edges() if u in critical_path and v in critical_path]
    other_edges = [e for e in G.edges() if e not in critical_edges]

    nx.draw_networkx_edges(G, pos, edgelist=other_edges, edge_color="black", width=2, arrowsize=20, style='dashed')
    nx.draw_networkx_edges(G, pos, edgelist=critical_edges, edge_color="red", width=3.5, arrowsize=20, style='solid')

    nx.draw_networkx_nodes(G, pos, node_color="lightblue", node_size=2500)
    nx.draw_networkx_labels(G, pos)

# Legend for critical and non-critical paths
    red_patch = mpatches.Patch(color='red', label='Critical Path')
    black_patch = mpatches.Patch(color='black', label='Non-Critical Path (Dashed)')
    plt.legend(handles=[red_patch, black_patch], loc='upper left', bbox_to_anchor=(1, 1))


    plt.title(" Critical Path Visualization")
    plt.axis('off')

    graph_path = tempfile.NamedTemporaryFile(delete=False, suffix=".png").name
    plt.savefig(graph_path)
    plt.close()

    # Generate Word Document
    doc = Document()
    doc.add_heading("Project Report", 0)

    format_table(doc, freelancers, " Freelancer Data")
    format_table(doc, tasks, " Task Data")
    format_cost_matrix_table(doc, cost_matrix[:num_freelancers, :num_tasks])
    format_table(doc, df_assignments, " Assignments (Hungarian Method)")
    format_table(doc, pd.DataFrame(schedule_data), " CPM Task Schedule")

    doc.add_heading("Critical Path Graph", level=2)
    doc.add_picture(graph_path, width=Inches(8))

    doc.add_heading(" Summary", level=2)
    doc.add_paragraph(f" Total Assignment Cost: ${total_cost:.2f}")
    if budget > 0:
        if total_cost > budget:
            doc.add_paragraph(f" Budget Exceeded by ${total_cost - budget:.2f}")
        else:
            doc.add_paragraph(f" Within Budget (${budget})")
    doc.add_paragraph(f" Critical Path: {' → '.join(critical_path)}")
    doc.add_paragraph(f" Project Duration: {project_duration} days")

    word_path = tempfile.NamedTemporaryFile(delete=False, suffix=".docx").name
    doc.save(word_path)

    return df_assignments, pd.DataFrame(schedule_data), graph_path, word_path

# Gradio UI
demo = gr.Interface(
    fn=optimize_and_schedule,
    inputs=[
        gr.File(label=" Upload Freelancer CSV"),
        gr.File(label=" Upload Tasks CSV"),
        gr.Number(label=" Enter Budget", value=0)
    ],
    outputs=[
        gr.Dataframe(label=" Assignment Results"),
        gr.Dataframe(label=" CPM Schedule"),
        gr.Image(label=" CPM Graph"),
        gr.File(label=" Download Full Report (Word)")
    ],
    title=" Freelancer Assignment + CPM Scheduler (Auto Report)",
    description="Upload your freelancer & task files and get all analytics in a downloadable Word report."
)

demo.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.2 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7139e17dd7428712b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
